In [10]:
from pyowm import OWM
from pyowm.weatherapi25.forecast import Forecast
from config import OWM_API_key as key, port, host
from pprint import pprint
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure, InvalidDocument, DuplicateKeyError
# import json
import os
from os import path


API_key = key
host = host
port = port
filename = 'resources/zip_list.csv'
global zipcode
owm = OWM(API_key) # the OWM object
global obs
global reception_time
global zid# = Zip_Location.get_ID()
global zlon# = location.get_lon()
global zlat# = location.get_lat()

In [2]:
# Zip_Weather_Data = owm.weather_at_zip_code('27606', 'us') # returns a Weather object corrosponding to that zipcode
# weather = Zip_Weather_Data
# Zip_Location
# location = obs.get_location() # returns a Location object corrosponding to the zipcode
# #  = Zip_Location
# zid = Zip_Location.get_ID()
# zlon = location.get_lon()
# zlat = location.get_lat()

In [11]:
def set_location(code):
    ''' Get the latitude and longitude corrosponding to the zip code. '''
    global obs, zlat, zlon
    obs = owm.weather_at_zip_code(f'{code}', 'us')
    location = obs.get_location()
    zlon = location.get_lon()
    zlat = location.get_lat()
    return

def current():
    ''' Dump the current weather to a json

    :params: '''
    global obs
    current = obs.to_JSON() # dump all that weather data to a json object
    return(current)

def five_day():
    ''' Get each weather forecast for the corrosponding zip code. '''
    global obs, zlat, zlon
    forecaster = owm.three_hours_forecast_at_coords(zlat, zlon)
    forecast = forecaster.get_forecast()
    five_day = forecast.to_JSON()
    return(five_day)

# YOU DON'T HAVE THE API ACCESS TO GET THIS TYPE OF FORECAST REQUEST
# def daily():
#     ''' Fetch the daily forecast. '''
#     forecaster = owm.daily_forecast_at_coords(zlat, zlon)
#     forecast = forecaster.get_forecast()
#     daily = forecast.to_JSON()
#     return(daily)
    
def check_db_access(str: host, port):
    '''A check that there is write access to the database'''
    client = MongoClient(host=host, port=port)
    try:
        # The ismaster command is cheap and does not require auth.
        client.admin.command('ismaster')
        print('Connection made')
    except ConnectionFailure:
        print("Server not available")
        return

    # check the database connections
        # Get a count of the number of databases at the connection (accessible through that port)
        # before attempting to add to it
    db_count_pre = len(client.list_database_names())
        # Add a database and collection
    db = client.test_db
    col = db.test_col

    # Insert something to the db
    post = {'name':'Chuck VanHoff',
           'age':'38',
           'hobby':'gardening'
           }
    col.insert_one(post)

        # Get a count of the databases after adding one
    db_count_post = len(client.list_database_names())

    if db_count_pre-db_count_post>=0:
        print('Your conneciton is flipped up')
    else:
        print('You have write access')

    client.drop_database(db)
    return(client)

def to_json(data, code):
    ''' Store the collected data as a json file in the case that the database
        is not connected or otherwise unavailable. 
    '''
    collection = data
    directory = os.getcwd()
    save_path = path.join(directory, 'Data', f'{code}.json')
    Data = open(save_path, 'a+')
    Data.write(collection)
    Data.close()
    return

def load(data):
    ''' Load the data to the database if possible, otherwise write to json file. '''
    client = check_db_access(host, port)
    if type(data) == dict:
        try:
            db = client.forcast
            col = db.code
            col.insert_one(data)
        except DuplicateKeyError:
            client.close()
            print('closed db connection')
            to_json(data, code)
            print('Wrote to json')
    else:
        print('data type is not dict')
        client.close()
        print('closed db connection')
    return

def write_list_to_file(zip_list, filename):
    """Write the zip codes to csv file.
    
    """
    with open(filename, "w") as z_list:
        z_list.write(",".join(str(z) for z in zip_list))
    return
        
def read_list_from_file(filename):
    """ Read the zip codes list from the csv file."""
    with open(filename, "r") as z_list:
        return z_list.read().strip().split(',')

def get_weather(codes):
    ''' Get the weather from the API and load it to the database. 
    
    :param codes: list of zip codes
    :type codes: list of strings
    '''
    for code in codes:
        data = {}
        print(code, data)
        set_location(code)
        data.update({'zipcode': code,
                     'current': current(),
                     'five_day': five_day(),
                    })
        load(data)
        client.close()
    return

In [12]:
# codes = ['27606', '27006', '92683']
codes = read_list_from_file(filename)
get_weather(codes)
# for code in codes:
#     data = {}
#     print(code, data)
#     set_location(code)
#     data.update({'zipcode': code,
#                  'current': current(),
#                  'five_day': five_day(),
#                 })
#     load(data)
#     pprint(data)
# print(data)

25911 {}
in load()
in db_check()
Connection made
You have write access
Inserted to database
closed db connection
27601 {}
in load()
in db_check()
Connection made
You have write access
Inserted to database
closed db connection
27602 {}
in load()
in db_check()
Connection made
You have write access
Inserted to database
closed db connection
27603 {}
in load()
in db_check()
Connection made
You have write access
Inserted to database
closed db connection
27604 {}
in load()
in db_check()
Connection made
You have write access
Inserted to database
closed db connection
27605 {}
in load()
in db_check()
Connection made
You have write access
Inserted to database
closed db connection
27606 {}
in load()
in db_check()
Connection made
You have write access
Inserted to database
closed db connection
27607 {}
in load()
in db_check()
Connection made
You have write access
Inserted to database
closed db connection
27608 {}
in load()
in db_check()
Connection made
You have write access
Inserted to database
clos

In [5]:
print(data)

{'zipcode': '92683', 'current': '{"reception_time": 1576978815, "Location": {"name": "Westminster", "coordinates": {"lon": -117.99, "lat": 33.75}, "ID": 0, "country": "US"}, "Weather": {"reference_time": 1576978802, "sunset_time": 1576975630, "sunrise_time": 1576939969, "clouds": 75, "rain": {}, "snow": {}, "wind": {"speed": 2.66, "deg": 136}, "humidity": 34, "pressure": {"press": 1016, "sea_level": null}, "temperature": {"temp": 288.94, "temp_kf": null, "temp_max": 292.15, "temp_min": 286.48}, "status": "Clouds", "detailed_status": "broken clouds", "weather_code": 803, "weather_icon_name": "04n", "visibility_distance": 16093, "dewpoint": null, "humidex": null, "heat_index": null}}', 'five_day': '{"interval": "3h", "reception_time": 1576978815, "Location": {"name": "Midway City", "coordinates": {"lon": -117.9893, "lat": 33.7447}, "ID": 5372902, "country": "US"}, "weathers": [{"reference_time": 1576983600, "sunset_time": 0, "sunrise_time": 0, "clouds": 100, "rain": {}, "snow": {}, "wind